In [63]:
## Description

> The notebook wrangles the twitter archive of WeRateDogs.

> WeRateDogs is a popular twitter channel that rates people's dogs on a scale of 10. They are known for their unique rating system wherein some dogs get rated on a scale higher than 10.


> The wrangling process is divided into the following parts:
>                                                                        
    1. Gathering 
    2. Assesing  
    3. Cleaning  
    4. Storing  
    5. Analysis  
    6. Visualization  
    

In [64]:
## Setup

In [65]:
# Importing general libraries

import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [66]:
## Gathering`

>The complete data for this project would be gathered from three different sources in three different file formats:  
 
> *Twitter archive enhanced*: a **csv** file on hand.  

> *Tweet image predictions*: a **tsv** file downloaded from a remote server.  

> *Tweet attributes*: a **txt** file extracted from a remote API. 

In [67]:
### Twitter archive enhanced

The file contains the tweet information of WeRateDogs up until 2017.The text field of the file has been used to extract some additional information such as the name of the dog considered and the the stage of the dog hence the name twitter-archive-enhanced.

Since the file is given to us, we just have to load it onto our workspace.

In [ ]:
#### Load

In [68]:
# Loading twitter-archive-enhanced onto the workspace

twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
#### Preview

In [69]:
# Preview

twitter_archive_enhanced.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [70]:
### Tweet image predictions

The images from the tweets of WeRateDogs was run through a neural network that classified breed of dogs. The image_predictions.tsv file contains the predictions from the neural network.

The file is stored on Udacity's servers. We would be using the requests library to extract the file from the servers.

In [71]:
#### Extract & store

In [72]:
# Extracting image_predictions.tsv and storing it in extracted_files.csv

import requests

url =  "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)

folder_name = 'extracted_files'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
with open(os.path.join(folder_name, url.split('/')[-1]),'wb') as file:
    file.write(response.content)

In [73]:
#### Load

In [74]:
# Loading image_predictions.tsv to workspace

tweet_image_predictions = pd.read_csv('extracted_files/image-predictions.tsv')

In [75]:
#### Preview

In [76]:
tweet_image_predictions.head()

,tweet_id\tjpg_url\timg_num\tp1\tp1_conf\tp1_dog\tp2\tp2_conf\tp2_dog\tp3\tp3_conf\tp3_dog
0,666020888022790149\thttps://pbs.twimg.com/medi...
1,666029285002620928\thttps://pbs.twimg.com/medi...
2,666033412701032449\thttps://pbs.twimg.com/medi...
3,666044226329800704\thttps://pbs.twimg.com/medi...
4,666049248165822465\thttps://pbs.twimg.com/medi...


In [77]:
### Tweet attributes

Since the twitter-archive-enhanced dataset is missing the retweet_count & favorite_count of tweets, we will be extracting the same by quering twitter's API with the tweet_id's of the tweets.

We will use python's tweepy library to interact with twitter's API and extract the required information. Since some tweets would be deleted from the the time the twitter_archive_enhanced dataset was prepared, some of of queries would fail.

We will store the info extracted from the successful queries and store it in a json file called tweet_attrs.
As we woud be catching the failed queries, the tweet_id's associated with them would be stored in a separate csv file called deleted_tweets

In [78]:
#### Extract

In [79]:
# Creating API Object

import tweepy


""" 
                  Store API Credential from twitter developer account.
                  Fill them in the empty strings below
                
      
"""

consumer_key    =    'O1HGcGIIgMqekydzMZbkKnOxD'
consumer_secret =    'Kw0WVgb3We57emyIMWVCBj14yd67mZAu2uuM7HoM6MRbMkvzaC'
access_token    =    '1410119509583691780-7qiMX7VhE8W4b2IazGnEKobrxUzRlI'
access_secret   =    'sqDMEKrU2WRFkslCDhPaBR366CMNl4qVLmhUlu7Zw7YvK'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [81]:
# Quering twiter API 


"""                          CAUTION: ThiS cell has a run time of 30 - 35 mins 
"""


"""  
               The script below queries the twitter API using the tweet_id from twitter_archive_enhanced 
               and writes the content of the API's json response to file: tweet_json.txt. 

               Simulataneously, it creates a list of all failed extractions containing the tweet_id and
               the error message.
               
"""



import json 

failed_extractions = []
itr = 1

with open('extracted_files/tweet_json.txt', 'w') as target_file:
    
    for id in twitter_archive_enhanced.tweet_id:
        
        print(f'Iteration: {itr}')
        print(f'Tweet ID: {id}')
        itr += 1
        
        try:

            tweet_attrs =  api.get_status(id, tweet_mode = 'extended')
            json.dump(tweet_attrs._json, target_file)
            target_file.write('\n')

            print('ID Extracted')
            
        except tweepy.TweepError as te:
            
            failed_extractions.append( { 'Id' : id,
                                         'Error' : te
                                       })
            
            print('ID Not Extracted')    

Iteration: 1
Tweet ID: 892420643555336193
ID Extracted
Iteration: 2
Tweet ID: 892177421306343426
ID Extracted
Iteration: 3
Tweet ID: 891815181378084864
ID Extracted
Iteration: 4
Tweet ID: 891689557279858688
ID Extracted
Iteration: 5
Tweet ID: 891327558926688256
ID Extracted
Iteration: 6
Tweet ID: 891087950875897856
ID Extracted
Iteration: 7
Tweet ID: 890971913173991426
ID Extracted
Iteration: 8
Tweet ID: 890729181411237888
ID Extracted
Iteration: 9
Tweet ID: 890609185150312448
ID Extracted
Iteration: 10
Tweet ID: 890240255349198849
ID Extracted
Iteration: 11
Tweet ID: 890006608113172480
ID Extracted
Iteration: 12
Tweet ID: 889880896479866881
ID Extracted
Iteration: 13
Tweet ID: 889665388333682689
ID Extracted
Iteration: 14
Tweet ID: 889638837579907072
ID Extracted
Iteration: 15
Tweet ID: 889531135344209921
ID Extracted
Iteration: 16
Tweet ID: 889278841981685760
ID Extracted
Iteration: 17
Tweet ID: 888917238123831296
ID Extracted
Iteration: 18
Tweet ID: 888804989199671297
ID Extracted
I

ID Extracted
Iteration: 147
Tweet ID: 863471782782697472
ID Extracted
Iteration: 148
Tweet ID: 863432100342583297
ID Extracted
Iteration: 149
Tweet ID: 863427515083354112
ID Extracted
Iteration: 150
Tweet ID: 863079547188785154
ID Extracted
Iteration: 151
Tweet ID: 863062471531167744
ID Extracted
Iteration: 152
Tweet ID: 862831371563274240
ID Extracted
Iteration: 153
Tweet ID: 862722525377298433
ID Extracted
Iteration: 154
Tweet ID: 862457590147678208
ID Extracted
Iteration: 155
Tweet ID: 862096992088072192
ID Extracted
Iteration: 156
Tweet ID: 861769973181624320
ID Not Extracted
Iteration: 157
Tweet ID: 861383897657036800
ID Extracted
Iteration: 158
Tweet ID: 861288531465048066
ID Extracted
Iteration: 159
Tweet ID: 861005113778896900
ID Extracted
Iteration: 160
Tweet ID: 860981674716409858
ID Extracted
Iteration: 161
Tweet ID: 860924035999428608
ID Extracted
Iteration: 162
Tweet ID: 860563773140209665
ID Extracted
Iteration: 163
Tweet ID: 860524505164394496
ID Extracted
Iteration: 164

ID Extracted
Iteration: 291
Tweet ID: 838150277551247360
ID Extracted
Iteration: 292
Tweet ID: 838085839343206401
ID Extracted
Iteration: 293
Tweet ID: 838083903487373313
ID Extracted
Iteration: 294
Tweet ID: 837820167694528512
ID Extracted
Iteration: 295
Tweet ID: 837482249356513284
ID Extracted
Iteration: 296
Tweet ID: 837471256429613056
ID Extracted
Iteration: 297
Tweet ID: 837366284874571778
ID Not Extracted
Iteration: 298
Tweet ID: 837110210464448512
ID Extracted
Iteration: 299
Tweet ID: 837012587749474308
ID Not Extracted
Iteration: 300
Tweet ID: 836989968035819520
ID Extracted
Iteration: 301
Tweet ID: 836753516572119041
ID Extracted
Iteration: 302
Tweet ID: 836677758902222849
ID Extracted
Iteration: 303
Tweet ID: 836648853927522308
ID Extracted
Iteration: 304
Tweet ID: 836397794269200385
ID Extracted
Iteration: 305
Tweet ID: 836380477523124226
ID Extracted
Iteration: 306
Tweet ID: 836260088725786625
ID Extracted
Iteration: 307
Tweet ID: 836001077879255040
ID Extracted
Iteration:

ID Not Extracted
Iteration: 434
Tweet ID: 820690176645140481
ID Not Extracted
Iteration: 435
Tweet ID: 820494788566847489
ID Not Extracted
Iteration: 436
Tweet ID: 820446719150292993
ID Not Extracted
Iteration: 437
Tweet ID: 820314633777061888
ID Not Extracted
Iteration: 438
Tweet ID: 820078625395449857
ID Not Extracted
Iteration: 439
Tweet ID: 820013781606658049
ID Not Extracted
Iteration: 440
Tweet ID: 819952236453363712
ID Not Extracted
Iteration: 441
Tweet ID: 819924195358416896
ID Not Extracted
Iteration: 442
Tweet ID: 819711362133872643
ID Not Extracted
Iteration: 443
Tweet ID: 819588359383371776
ID Not Extracted
Iteration: 444
Tweet ID: 819347104292290561
ID Not Extracted
Iteration: 445
Tweet ID: 819238181065359361
ID Not Extracted
Iteration: 446
Tweet ID: 819227688460238848
ID Not Extracted
Iteration: 447
Tweet ID: 819015337530290176
ID Not Extracted
Iteration: 448
Tweet ID: 819015331746349057
ID Not Extracted
Iteration: 449
Tweet ID: 819006400881917954
ID Not Extracted
Iterati

ID Extracted
Iteration: 573
Tweet ID: 801285448605831168
ID Extracted
Iteration: 574
Tweet ID: 801167903437357056
ID Extracted
Iteration: 575
Tweet ID: 801127390143516673
ID Extracted
Iteration: 576
Tweet ID: 801115127852503040
ID Extracted
Iteration: 577
Tweet ID: 800859414831898624
ID Extracted
Iteration: 578
Tweet ID: 800855607700029440
ID Extracted
Iteration: 579
Tweet ID: 800751577355128832
ID Extracted
Iteration: 580
Tweet ID: 800513324630806528
ID Extracted
Iteration: 581
Tweet ID: 800459316964663297
ID Extracted
Iteration: 582
Tweet ID: 800443802682937345
ID Extracted
Iteration: 583
Tweet ID: 800388270626521089
ID Extracted
Iteration: 584
Tweet ID: 800188575492947969
ID Extracted
Iteration: 585
Tweet ID: 800141422401830912
ID Extracted
Iteration: 586
Tweet ID: 800018252395122689
ID Extracted
Iteration: 587
Tweet ID: 799774291445383169
ID Extracted
Iteration: 588
Tweet ID: 799757965289017345
ID Extracted
Iteration: 589
Tweet ID: 799422933579902976
ID Extracted
Iteration: 590
Twe

...

Rate limit reached. Sleeping for: 274


ID Extracted
Iteration: 1909
Tweet ID: 674436901579923456
ID Extracted
Iteration: 1910
Tweet ID: 674422304705744896
ID Extracted
Iteration: 1911
Tweet ID: 674416750885273600
ID Extracted
Iteration: 1912
Tweet ID: 674410619106390016
ID Extracted
Iteration: 1913
Tweet ID: 674394782723014656
ID Extracted
Iteration: 1914
Tweet ID: 674372068062928900
ID Extracted
Iteration: 1915
Tweet ID: 674330906434379776
ID Extracted
Iteration: 1916
Tweet ID: 674318007229923329
ID Extracted
Iteration: 1917
Tweet ID: 674307341513269249
ID Extracted
Iteration: 1918
Tweet ID: 674291837063053312
ID Extracted
Iteration: 1919
Tweet ID: 674271431610523648
ID Extracted
Iteration: 1920
Tweet ID: 674269164442398721
ID Extracted
Iteration: 1921
Tweet ID: 674265582246694913
ID Extracted
Iteration: 1922
Tweet ID: 674262580978937856
ID Extracted
Iteration: 1923
Tweet ID: 674255168825880576
ID Extracted
Iteration: 1924
Tweet ID: 674082852460433408
ID Extracted
Iteration: 1925
Tweet ID: 674075285688614912
ID Extracted
I

ID Extracted
Iteration: 2051
Tweet ID: 671497587707535361
ID Extracted
Iteration: 2052
Tweet ID: 671488513339211776
ID Extracted
Iteration: 2053
Tweet ID: 671486386088865792
ID Extracted
Iteration: 2054
Tweet ID: 671485057807351808
ID Extracted
Iteration: 2055
Tweet ID: 671390180817915904
ID Extracted
Iteration: 2056
Tweet ID: 671362598324076544
ID Extracted
Iteration: 2057
Tweet ID: 671357843010908160
ID Extracted
Iteration: 2058
Tweet ID: 671355857343524864
ID Extracted
Iteration: 2059
Tweet ID: 671347597085433856
ID Extracted
Iteration: 2060
Tweet ID: 671186162933985280
ID Extracted
Iteration: 2061
Tweet ID: 671182547775299584
ID Extracted
Iteration: 2062
Tweet ID: 671166507850801152
ID Extracted
Iteration: 2063
Tweet ID: 671163268581498880
ID Extracted
Iteration: 2064
Tweet ID: 671159727754231808
ID Extracted
Iteration: 2065
Tweet ID: 671154572044468225
ID Extracted
Iteration: 2066
Tweet ID: 671151324042559489
ID Extracted
Iteration: 2067
Tweet ID: 671147085991960577
ID Extracted
I

ID Extracted
Iteration: 2193
Tweet ID: 668932921458302977
ID Extracted
Iteration: 2194
Tweet ID: 668902994700836864
ID Extracted
Iteration: 2195
Tweet ID: 668892474547511297
ID Extracted
Iteration: 2196
Tweet ID: 668872652652679168
ID Extracted
Iteration: 2197
Tweet ID: 668852170888998912
ID Extracted
Iteration: 2198
Tweet ID: 668826086256599040
ID Extracted
Iteration: 2199
Tweet ID: 668815180734689280
ID Extracted
Iteration: 2200
Tweet ID: 668779399630725120
ID Extracted
Iteration: 2201
Tweet ID: 668655139528511488
ID Extracted
Iteration: 2202
Tweet ID: 668645506898350081
ID Extracted
Iteration: 2203
Tweet ID: 668643542311546881
ID Extracted
Iteration: 2204
Tweet ID: 668641109086707712
ID Extracted
Iteration: 2205
Tweet ID: 668636665813057536
ID Extracted
Iteration: 2206
Tweet ID: 668633411083464705
ID Extracted
Iteration: 2207
Tweet ID: 668631377374486528
ID Extracted
Iteration: 2208
Tweet ID: 668627278264475648
ID Extracted
Iteration: 2209
Tweet ID: 668625577880875008
ID Extracted
I

ID Extracted
Iteration: 2335
Tweet ID: 666293911632134144
ID Extracted
Iteration: 2336
Tweet ID: 666287406224695296
ID Extracted
Iteration: 2337
Tweet ID: 666273097616637952
ID Extracted
Iteration: 2338
Tweet ID: 666268910803644416
ID Extracted
Iteration: 2339
Tweet ID: 666104133288665088
ID Extracted
Iteration: 2340
Tweet ID: 666102155909144576
ID Extracted
Iteration: 2341
Tweet ID: 666099513787052032
ID Extracted
Iteration: 2342
Tweet ID: 666094000022159362
ID Extracted
Iteration: 2343
Tweet ID: 666082916733198337
ID Extracted
Iteration: 2344
Tweet ID: 666073100786774016
ID Extracted
Iteration: 2345
Tweet ID: 666071193221509120
ID Extracted
Iteration: 2346
Tweet ID: 666063827256086533
ID Extracted
Iteration: 2347
Tweet ID: 666058600524156928
ID Extracted
Iteration: 2348
Tweet ID: 666057090499244032
ID Extracted
Iteration: 2349
Tweet ID: 666055525042405380
ID Extracted
Iteration: 2350
Tweet ID: 666051853826850816
ID Extracted
Iteration: 2351
Tweet ID: 666050758794694657
ID Extracted
I

In [82]:
# Handling Failed Extractions

"""
                The script works with tweet_id's associated with failed extractions(deleted tweets)

                The tweet_id, alongside null initialzed values for retweet_count and favorite count
                are stored in a dataframe called deleted tweets which is then exported to a file
                deleted_tweets.txt.
                        
"""

extractions_fail = []

for error_info in failed_extractions:
    
    tweet_id = error_info['Id']
    
    extractions_fail.append( {
                             'tweet_id' : error_info['Id'],
                             'retweet_count' :'N/A',
                             'favorite_count': 'N/A'
                              })
    
deleted_tweets = pd.DataFrame(extractions_fail, columns = ['tweet_id','retweet_count','favorite_count'])
deleted_tweets.to_csv('extracted_files/deleted_tweets.csv')

In [83]:
#### Load & store

In [84]:
"""
                            This script:

                            1. Reads json data from tweet_json.txt
                            2. Extracts the required attributes from json
                            3. Stores them to a dataframe
                            4. Exports the dataframe to a csv file
                                
"""



with open('extracted_files/tweet_json.txt', 'r') as src_file:
    
    json_data = src_file.read()
    json_data = json_data.split('\n')
    json_data.pop()

    
tweet_attrs = []
missing_attrs = []

for tweet_data in json_data:
    
    
    tweet_data = json.loads(tweet_data)

    tweet_id = tweet_data['id']
    retweet_count = tweet_data['retweet_count']
    favorite_count = tweet_data['favorite_count']

    tweet_attrs.append( { 
                          'tweet_id' : tweet_id,
                          'retweet_count' : retweet_count,
                          'favorite_count' : favorite_count
                         } )

    

tweet_attrs = pd.DataFrame(tweet_attrs, columns = ['tweet_id','retweet_count','favorite_count'])
tweet_attrs.to_csv('extracted_files/tweet_attrs.csv')

In [85]:
#### Previews

In [86]:
# Preview tweet_attrs

tweet_attrs.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7234,34638
1,892177421306343426,5418,30020
2,891815181378084864,3579,22601
3,891689557279858688,7435,37893
4,891327558926688256,7991,36173


In [87]:
# Preview deleted_tweets

deleted_tweets.head()

,tweet_id,retweet_count,favorite_count
0,888202515573088257,N/A,N/A
1,873697596434513921,N/A,N/A
2,872668790621863937,N/A,N/A
3,872261713294495745,N/A,N/A
4,869988702071779329,N/A,N/A


In [ ]:
##  Assessing

In [ ]:
### Visual assesment 

In [ ]:
### Programmatic assesment

In [ ]:
### Documenting Issues

In [ ]:
#### Quality issues

In [ ]:
#### Tidiness issues

In [4]:
## Cleaning

In [ ]:
### Quality issues

In [ ]:
### Tidiness issues 

In [ ]:
## Storing

In [ ]:
### Twitter archive master to a csv file

In [ ]:
### Twitter arhive master to a sqlite database

In [5]:
## Analysis and visualization

In [ ]:
### Most tweeted dog stage

In [ ]:
### Average popularity rating of WeRateDogs

In [ ]:
### Popularity trend of WeRateDogs